In [2]:
import numpy as np 
import pandas as pd 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/Shareddrives/Hackathon/AMOC/sentiment140.zip -d /content/
!mv /content/'training.1600000.processed.noemoticon.csv' /content/dataset.csv

Archive:  /content/drive/Shareddrives/Hackathon/AMOC/sentiment140.zip
  inflating: /content/training.1600000.processed.noemoticon.csv  


# Loading the Dataset, We use engine=python as loading was giving error ( not able to read large sentences ) 

In [ ]:
data=pd.read_csv("/content/dataset.csv", encoding = "ISO-8859-1", engine="python")

In [ ]:
data.tail(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:


data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


# modifying data columns for better understanding

In [ ]:
data.columns = ["label", "time", "date", "query", "username", "text"]


In [ ]:
data

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


# Checking if values are null


In [ ]:
data.isnull().sum()

label       0
time        0
date        0
query       0
username    0
text        0
dtype: int64

# Only text and label should be used as conclusion

In [ ]:
data=data[['label','text']]

In [ ]:
data['text']=data['text'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# We use Nltk library for text analysis
Various procedures involved are :-

# i) Cleaning DATA


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Importing stop words from nltk 

In [ ]:
data.label.value_counts()

4    800000
0    799999
Name: label, dtype: int64

# words used as stop words

In [ ]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')

FIRST 5 STOPWORDS

In [ ]:
stopwords_list[0:5]

['i', 'me', 'my', 'myself', 'we']

In [ ]:
data['label'][data['label']==4]=1
data_pos = data[data['label'] == 1]
data_neg = data[data['label'] == 0]

In [ ]:
data_pos = data_pos.iloc[:int(40000)]
data_neg = data_neg.iloc[:int(40000)]

In [ ]:
data = pd.concat([data_pos, data_neg])

# Removing stopwords from dataset

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()

799999    ['love', 'healthuandpets', 'u', 'guys', 'r', '...
800000    ['im', 'meeting', 'one', 'besties', 'tonight',...
800001    ['darealsunisakim', 'thanks', 'twitter', 'add'...
800002    ['sick', 'really', 'cheap', 'hurts', 'much', '...
800003              ['lovesbrooklyn', 'effect', 'everyone']
Name: text, dtype: object

# Cleaning and removing punctutions

In [ ]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
punctuations_list[0:]

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
data['text'].tail()

39995    mittec problems literally spent hours cleaning...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# Cleaning and removing URL's

In [ ]:
import re
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(data))
data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))
data['text'].tail()

39995    mittec problems literally spent hours cleaning...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# Removing Numeric characters

In [ ]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))
data['text'].tail()

39995    mittec problems literally spent hours cleaning...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# ii) TOKENIZING 
Tokenization is the process of converting data into smaller sub parts for better analysis ( Eg: breaking words in sentence)

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)
data.head()

,label,text
799999,1,"[love, healthuandpets, u, guys, r, best]"
800000,1,"[im, meeting, one, besties, tonight, cant, wai..."
800001,1,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,1,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,1,"[lovesbrooklyn, effect, everyone]"


# iii) Applying Stemming
We reduce words to their root form

In [ ]:
from nltk.stem import PorterStemmer 

In [ ]:
stemer = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [stemer.stem(word) for word in data]
    return data

data['text']= data['text'].apply(lambda x: stemming_on_text(x))

In [ ]:
data.head()

,label,text
799999,1,"[love, healthuandpets, u, guys, r, best]"
800000,1,"[im, meeting, one, besties, tonight, cant, wai..."
800001,1,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,1,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,1,"[lovesbrooklyn, effect, everyone]"


# iv) Applying Lemmatization
It is same as stemming but it also looks for dictionary relation while stemming is dependent on part of speech

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))

In [ ]:
data.head()

,label,text
799999,1,"[love, healthuandpets, u, guys, r, best]"
800000,1,"[im, meeting, one, besties, tonight, cant, wai..."
800001,1,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,1,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,1,"[lovesbrooklyn, effect, everyone]"


Now data cleaning is done

In [ ]:
input=data['text']
output=data['label']




In [ ]:
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:

max_len = 1000
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(input)
sequences = tok.texts_to_sequences(input)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix.shape

(80000, 1000)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, output, test_size=0.3, random_state=2)

In [ ]:
def tensorflow_based_model():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(2000,50,input_length=max_len)(inputs) 
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer) 
    layer = Activation('relu')(layer) 
    layer = Dropout(0.5)(layer) 
    layer = Dense(1,name='out_layer')(layer) 
    layer = Activation('sigmoid')(layer) 
    model = Model(inputs=inputs,outputs=layer) 
    return model 

In [ ]:
model = tensorflow_based_model() 
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])  

In [ ]:
history=model.fit(X_train,Y_train,batch_size=80,epochs=6, validation_split=0.1)
print('Training finished !!')

Epoch 1/6
630/630 [==============================] - 482s 762ms/step - loss: 0.5836 - accuracy: 0.6824 - val_loss: 0.5029 - val_accuracy: 0.7525
Epoch 2/6
630/630 [==============================] - 481s 764ms/step - loss: 0.6307 - accuracy: 0.7018 - val_loss: 0.5037 - val_accuracy: 0.7523
Epoch 3/6
630/630 [==============================] - 490s 778ms/step - loss: 0.4886 - accuracy: 0.7667 - val_loss: 0.5030 - val_accuracy: 0.7595
Epoch 4/6
630/630 [==============================] - 492s 781ms/step - loss: 0.4937 - accuracy: 0.7594 - val_loss: 0.5140 - val_accuracy: 0.7541
Epoch 5/6
630/630 [==============================] - 490s 778ms/step - loss: 0.4806 - accuracy: 0.7729 - val_loss: 0.5103 - val_accuracy: 0.7418
Epoch 6/6
630/630 [==============================] - 484s 769ms/step - loss: 0.4776 - accuracy: 0.7725 - val_loss: 0.5040 - val_accuracy: 0.7543
Training finished !!


In [ ]:
accr1 = model.evaluate(X_test,Y_test) #we are starting to test the model here

750/750 [==============================] - 91s 121ms/step - loss: 0.5047 - accuracy: 0.7573


In [ ]:
y_pred=model.predict(X_test)

In [5]:
y_pred = model.predict(X_test) #getting predictions on the trained model
y_pred = (y_pred > 0.5) 

NameError: ignored